# ChatGPT Annotation

### Import required libraries

In [1]:
from openai import OpenAI
import pandas as pd
from dotenv import load_dotenv
from time import sleep
import os

load_dotenv()

True

### ChatGPT Annotation

In [11]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
# TODO: Separate this data cleaning part in its own process
df = pd.read_csv("2023-01-31_skynews_openai_result.csv", sep=';')
print(df.shape)
df.dropna(subset=['Tags'], inplace=True)
df_first = df.drop(df[df['Tags'] == "['N/A']"].index)
df_second = df_first.drop(df_first[df_first['Tags'] == "[]"].index)
print(df_second.shape)
df_second.to_csv("2023-01-31_guardian_clean.csv", sep=';')

In [12]:
# TODO: clean?
df = pd.read_csv("2022-01_skynews_articles_clean_02.csv", sep=';')
df = df.reset_index()
print(df.shape)

(295, 10)


In [ ]:
# TODO: test if this works (should)
system_message = "You are an excellent annotator. You will be provided with a title in between <titleStart> and <titleEnd> and one or multiple tags in between <tagStart> and <tagEnd>. Both the title and tags belong to a news article. Your task is to classify to which of the following categories the article most likely belongs to. However, you are only allowed to pick categories from the following list: 'Category-1: politics', 'Category-2: business and economy', 'Category-3: environment', 'Category-4: sports', 'Category-5: entertainment and culture', 'Category-6: science and technology' or 'Category-7: health'. Return the number of only one of the provided categories."

for counter, (index, row) in enumerate(df.iterrows(), start=1):
    print(counter)
    title = row['Title']
    tags = row['Tags'][1:-1]
    user_message = f"<titleStart> {title} <titleEnd>; <tagStart> {tags} <tagEnd>"

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        temperature=0,
        max_tokens=256
    )

    category = response.choices[0].message.content
    print(category)
    
    # Following timeout is only needed for the free tier (uncomment if needed)
    # sleep(21)
    df.loc[index, 'Chat GPT'] = category

df.to_csv("2022-01_skynews_articles_annotated.csv", sep=';')